In [9]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os, string

In [10]:
sz = 200
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# third convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
#units changed 128 to 400 - 500 - 200
classifier.add(Dense(units=200, activation='relu'))
classifier.add(Dropout(0.40))
#units changed 96 to 800 - 300 - 150
classifier.add(Dense(units=150, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=26, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


In [11]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 97, 97, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 32)      

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=10, # rotate the image 30 degrees
        width_shift_range=0.1, # Shift the pic width by a max of 10%
        height_shift_range=0.1, # Shift the pic height by a max of 10%
        rescale=1/255, # Rescale the image by normalzing it.
        shear_range=0.2, # Shear means cutting away part of the image (max 20%)
        zoom_range=0.2, # Zoom in by 20% max
        horizontal_flip=True, # Allo horizontal flipping
        fill_mode='nearest' # Fill in missing pixels with the nearest filled value
        )
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.15,
                                  zoom_range=0.1,
                                  width_shift_range=0.15,
                                  height_shift_range=0.1,
                                  fill_mode='nearest')

In [13]:
training_set = train_datagen.flow_from_directory('./ProcessedData/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('./ProcessedData/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 13000 images belonging to 26 classes.
Found 5200 images belonging to 26 classes.


In [14]:
training_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [15]:
test_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [16]:
classifier.fit(
        training_set,
        steps_per_epoch=449, 
        epochs=10,
        validation_data=test_set,
        validation_steps=190)

Epoch 1/10
449/449 [==============================] - 104s 232ms/step - loss: 2.4120 - accuracy: 0.2708 - val_loss: 1.0723 - val_accuracy: 0.6779
Epoch 2/10
449/449 [==============================] - 104s 232ms/step - loss: 0.9897 - accuracy: 0.6728 - val_loss: 0.5101 - val_accuracy: 0.8389
Epoch 3/10
449/449 [==============================] - 105s 233ms/step - loss: 0.6347 - accuracy: 0.7837 - val_loss: 0.3672 - val_accuracy: 0.8900
Epoch 4/10
449/449 [==============================] - 104s 232ms/step - loss: 0.4692 - accuracy: 0.8403 - val_loss: 0.2700 - val_accuracy: 0.9116
Epoch 5/10
449/449 [==============================] - 107s 237ms/step - loss: 0.3833 - accuracy: 0.8717 - val_loss: 0.2697 - val_accuracy: 0.9216
Epoch 6/10
449/449 [==============================] - 120s 268ms/step - loss: 0.3162 - accuracy: 0.8904 - val_loss: 0.2517 - val_accuracy: 0.9358
Epoch 7/10
449/449 [==============================] - 108s 241ms/step - loss: 0.2656 - accuracy: 0.9143 - val_loss: 0.2471 -

In [43]:
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
#Importing testing image
test_file = './ProcessedData/Test/A/90.jpg'
test_img = image.load_img(test_file,target_size=(200, 200))
test_img = image.img_to_array(test_img)
test_img = cv2.cvtColor(np.array(test_img),cv2.COLOR_RGB2GRAY)

In [44]:
test_img.shape

(200, 200)

In [45]:
test_img = np.expand_dims(test_img, axis=0)
test_img = test_img/255

In [46]:
prediction_prob = classifier.predict(test_img)

In [47]:
prediction_prob

array([[9.99991417e-01, 1.79025143e-12, 2.81640519e-06, 5.67303232e-07,
        1.28991115e-14, 1.53844223e-14, 7.08355033e-13, 4.97730654e-13,
        4.67223428e-13, 2.47567857e-18, 2.38973077e-11, 1.37081169e-13,
        1.16473443e-12, 3.56667109e-14, 3.60343421e-07, 3.24990174e-10,
        1.91790452e-14, 9.53258219e-12, 5.53291898e-07, 4.34579852e-06,
        2.26282690e-08, 1.50572541e-10, 3.17505072e-12, 1.12655163e-10,
        2.83158474e-09, 5.51347332e-14]], dtype=float32)

In [48]:
np.argmax(prediction_prob)

0

In [49]:
classes = list(string.ascii_uppercase)

In [50]:
classes[np.argmax(prediction_prob)]

'A'

In [25]:
#saving the model
import os
if not os.path.exists("./Model"):
    os.mkdir("./Model")
classifier.save("./Model/Sign_classifier.h5")

In [26]:
classifier_Json = classifier.to_json()
with open("./Model/classifier_json.json", "w") as json_file:
    json_file.write(classifier_Json)
print('json model saved')
classifier.save_weights('./Model/classifier_weights.h5')
print('Weights saved')

json model saved
Weights saved


# Classifier Building - Completed